In [70]:
import pandas as pd
import numpy as np
from datetime import datetime

In [71]:
TC_File_name = 'CstAp_CANMGT_Integration_Testcase_v3.6.xlsm'

In [72]:
df = pd.read_excel(TC_File_name,sheet_name = 'TestDesign')
# workbook = openpyxl.load_workbook('CstAp_CANMGT_Integration_Testcase_v3.6.xlsm', data_only = True)  오래걸림
df111 = pd.read_excel(TC_File_name,sheet_name = 'TestDataConvert')

c:\Python\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [73]:
# Input, output 찾고 인덱스 지정 -> input ouput 데이터 분리시키기 위함
index_in = df.columns.get_loc('Inputs')
index_out = df.columns.get_loc('Expected Outputs')
index_act = df.columns.get_loc('Actual Outputs')

#input 개수, output 개수
numinput = df111.iloc[4,1]
numoutput = df111.iloc[4,4]

# TC, input, ouput, stepdata 모음
TCdata = df.iloc[:,0]
P= TCdata.dropna()
inputdata = df.iloc[:,index_in:index_out]
outputdata = df.iloc[:,index_out:index_act]
stepdata = df.iloc[:,6]

# inputID 모음
inputID = df.iloc[0,index_in:index_out]
inputID = np.insert(inputID,0,'<input>')
inputID = np.insert(inputID,0,'')

# outputID 모음
outputID = df.iloc[0,index_out:index_act]
outputID = np.insert(outputID,0,'<expect>')


# CASEID 위치 찾기
target_column_index = 0
target_column = df.iloc[:, target_column_index]

num = target_column.dropna().index.tolist()


In [74]:
def tempd_noid():

    #TCID 모음
    q=[]


    for i in range(1,len(num)):
        q.append(TCdata[num[i]:num[i+1]])
        if i ==(len(num)-2):
            q.append(TCdata[num[i+1]:len(TCdata)])
            break

##########################################################
    #Step 모음
    w=[]

    for i in range(1,len(num)):
        w.append(stepdata[num[i]:num[i+1]])
        if i ==(len(num)-2):
            w.append(stepdata[num[i+1]:len(TCdata)])
            break
##########################################################
        
    #input 모음
    e=[]

    for i in range(1,len(num)):
        e.append(inputdata[num[i]:num[i+1]])
        if i ==(len(num)-2):
            e.append(inputdata[num[i+1]:len(TCdata)])
            break

    # 빈칸이 있다면 이전스텝 Input복사
    for i in range(0,len(num)-1):
        e[i].fillna(method='ffill', inplace = True)

##########################################################
    #output 모음
    r=[]

    for i in range(1,len(num)):
        r.append(outputdata[num[i]:num[i+1]])
        if i ==(len(num)-2):
            r.append(outputdata[num[i+1]:len(TCdata)])
            break

    # 빈칸이 있다면 이전스텝 output복사
    for i in range(0,len(num)-1):
        r[i].fillna(method='ffill', inplace = True)

##########################################################

    #step, inputs, outputs 합치기, 실제데이터에는 TCID는 굳이 쓰이지 않음
    tempd_noid = []
    for i in range(0,len(num)-1):
        tempd_noid.append(pd.concat([w[i],e[i],r[i]],axis=1, ignore_index=False))

    return tempd_noid

a = tempd_noid()

C:\Users\user\AppData\Local\Temp\ipykernel_25188\1034429063.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  e[i].fillna(method='ffill', inplace = True)
C:\Users\user\AppData\Local\Temp\ipykernel_25188\1034429063.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e[i].fillna(method='ffill', inplace = True)
C:\Users\user\AppData\Local\Temp\ipykernel_25188\1034429063.py:49: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  r[i].fillna(method='ffill', inplace = True)
C:\Users\user\AppData\Local\Temp\ipykernel_25188\1034429063.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame


In [75]:
arr = []
steploop = [[]for _ in range(len(a))]


for i in range(0,len(a)):
    arr.append(a[i].values)
    for j in range(len(arr[i])):
        if j == (len(arr[i])-1):
            steploop[i].append(1)
            break
        steploop[i].append(arr[i][j+1,0]-arr[i][j,0])

    

    
stepinout = [[]for _ in range(len(a))]

for i in range(len(a)):
    for j in range(len(steploop[i])):
        count = steploop[i][j]
        for _ in range(count):
            stepinout[i].append(arr[i][j,:])

##########################################################
xx=[]
xxx=[]

for i in range(len(a)):
    xx.append(pd.DataFrame(stepinout[i]))
    xxx.append(xx[i].drop(xx[i].columns[0], axis=1))  #첫열 step열 짜름 , csv쓸 떄 필요하지 않음

##########################################################

row  = [[]for _ in range(len(a))]

for i in range(len(a)):

    for j  in range(0,arr[i][-1,0]):            # TCID 제외하고 step부터하기위해 0부터가 아닌 1부터
        row[i].append(xxx[i].iloc[j, 0:].values)

    # arr[-1.0] -> arr[0][-1,0]
    


In [76]:
#파일이름 읽기
CSVname = df111.iloc[2,1]

# 현재 시간을 얻기
current_time = datetime.now()

# 원하는 포맷으로 현재 시간 출력
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")


In [100]:



for k in range(0,len(a)):

    # aa = pd.DataFrame(index=range(1,400), columns=range(1,100))
    aa = pd.DataFrame(index = range(1,len(row[k])+4), columns=range(1,numinput+numoutput+4))    # 빈데이터 프레임 생성 (행: 스텝 수 + 3   /   열 = input개수 + output개수 + 3)

    # A1, A2에 CASE ID 및 시간 등 정보 기입
    aa.at[1,1] = f'TestCase Generation For CodeScroll Controller Tester by SL user at {formatted_time}'
    aa.at[2,1] = f'Name = {CSVname}_test{k}.csv     From = {P[num[k+1]]}     Of =csv_write.xlsm'


    # A3에 inputID, OuputID 기입
    for i in range(0,len(inputID)):
        aa.at[3,i+1] = inputID[i]
        if i == (len(inputID)-1):
            for j in range(0, len(outputID)):
                aa.at[3,i+1+j+1] = outputID[j]

    # A4부터 데이터 기입

    for i in range(0,len(row[k])):
        aa.at[i+4,1] = i+1
        for j in range(0,len(row[k][i])):

            if (j >= (index_out-index_in)):
                aa.at[4+i,(index_out-index_in)+3]= np.nan
                aa.at[4+i,j+4] = row[k][i][j]
                
            else :  # index_out-index_in
                aa.at[4+i,j+3] = row[k][i][j]

    #nan값 NAN으로 바꾸기
    # aa[aa == 'nan'] = np.nan

    bb = aa.iloc[3:]
    # cc = bb.fillna(-1)
    dd = bb.astype('Int64')
    # ee = dd.replace(-1,np.nan, inplace=False) #inplace = True하면 실수값이 반환됨...
    result_df = pd.concat([aa.iloc[:3], dd], ignore_index=False)

# # 실수로 저장된 값들 정수로 바꾸기
#     for i in range(0, aa.index[-1]):
#         for j in range(0, aa.columns[-1]):
#             if (pd.isna(aa.iloc[i,j]) or type(aa.iloc[i,j])==str):
#                 continue
#             aa.iloc[i,j] = int(aa.iloc[i,j])
        


    result_df.to_csv(f"{CSVname}_test{k}.csv", index=False, header=0)
